In [ ]:

###########################################################################################################################################################
# Citations:
# 1) Charan, G. (2020). Sensing-Aided-Drone-Beam-Prediction [Repository]. GitHub. https://github.com/gourangc/Sensing-Aided-Drone-Beam-Prediction
# 2) Cheng, R. (2019). LeNet-5-Implementation-Using-Pytorch [Repository]. GitHub. https://github.com/lychengrex/LeNet-5-Implementation-Using-Pytorch
###########################################################################################################################################################

In [2]:
from PIL import Image
import numpy as np
import os
import numpy as np
import pandas as pd
from natsort import natsorted
import re
import os
import os
import random
import pandas as pd
import torch
import numpy as np
import random
# from skimage import io
import skimage
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
import torch.nn.functional as F
import os
import datetime
import sys
import torch as t
import torch.cuda as cuda
import torch.optim as optimizer
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transf
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, transform
from scipy import io
from torchsummary import summary
import csv
import os
from google.colab import drive
drive.mount('/content/gdrive')

# Set the scenario number
scenario_numx = 7
scenario_num = scenario_numx

all_cars = True

# Set the working directory appropriately in this variable
directory = '/content/gdrive/MyDrive/Dataset/scenario' + str(scenario_numx)
directory = directory + '/'

# ! git clone https://github.com/gourangc/Vision-Position-Beam-Prediction.git

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Datafeed

In [ ]:
########################################################
#  Generate the train, test and validation data first. 
########################################################

In [4]:
# Charan, G. (2020). Sensing-Aided-Drone-Beam-Prediction [Repository]. GitHub. https://github.com/gourangc/Sensing-Aided-Drone-Beam-Prediction

'''
A data feeding class. It generates a list of data samples, each of which is
a tuple of a string (image path) and an integer (beam index), and it defines
a data-fetching method.
Author: Gouranga Charan
Nov. 2020
'''


############### Create data sample list #################
def create_samples(root, shuffle=False, nat_sort=False):
    f = pd.read_csv(root)
    data_samples = []
    pred_val = []
    for idx, row in f.iterrows():
        img_paths = row.values
        data_samples.append(img_paths)
    
    # print(data_samples)
    return data_samples
#############################################################

class DataFeed(Dataset):
    '''
    A class retrieving a tuple of (image,label). It can handle the case
    of empty classes (empty folders).
    '''
    def __init__(self,root_dir, nat_sort = False, transform=None, init_shuflle = True):
        self.root = root_dir
        self.samples = create_samples(self.root,shuffle=init_shuflle,nat_sort=nat_sort)
        self.transform = transform


    def __len__(self):
        return len( self.samples )

    def __getitem__(self, idx):
        sample = self.samples[idx]
        img = skimage.io.imread(directory +sample[5][2:])
        if self.transform:
            img = self.transform(img)
        label = sample[7]
        return (img,label)

# Buildnet

In [5]:

# Cheng, R. (2019). LeNet-5-Implementation-Using-Pytorch [Repository]. GitHub. https://github.com/lychengrex/LeNet-5-Implementation-Using-Pytorch

class LeNet(nn.Module):

    # network structure
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 64)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        return np.prod(size)     






#Main script

In [6]:
# Charan, G. (2020). Sensing-Aided-Drone-Beam-Prediction [Repository]. GitHub. https://github.com/gourangc/Sensing-Aided-Drone-Beam-Prediction
'''
Main script
Author: Gouranga Charan
Nov. 2020
'''

isExists = os.path.exists(directory + 'checkpoint')
if not isExists:
    os.makedirs(directory + 'checkpoint') 
    
isExists = os.path.exists(directory + 'saved_analysis_file')
if not isExists:
    os.makedirs(directory+'/saved_analysis_file')        

# Hyper-parameters
batch_size = 32
val_batch_size = 1
lr = 1e-2
# decay = 1e-4
image_grab = False
num_epochs = 1
train_size = [1]

# Data pre-processing:
img_resize = transf.Resize((28, 28))
# img_norm = transf.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
proc_pipe = transf.Compose(
    [transf.ToPILImage(),
      img_resize,
      transf.ToTensor()]
)

val_dir = '/content/scenario7_test.csv'

# train_dir=val_dir


val_loader = DataLoader(DataFeed(val_dir, transform=proc_pipe),
                        batch_size=val_batch_size,
                        shuffle=True)
now = datetime.datetime.now()



val_acc = []
with cuda.device(0):
    top_1 = np.zeros( (1,len(train_size)) )
    top_2 = np.zeros( (1,len(train_size)) )
    top_3 = np.zeros( (1,len(train_size)) )
    acc_loss = 0
    itr = []
    for idx, n in enumerate(train_size):
        print('```````````````````````````````````````````````````````')
        print('Training size is {}'.format(n))
        # Build the network:
        net = LeNet()
        net = net.cuda()
        checkpoint_path = '/content/checkpoint_yolov7_scenario7'
        net.load_state_dict(t.load(checkpoint_path))
        net.eval()
        layers = list(net.children())
        summary(net.cuda(), (1, 28, 28))

        #  Optimization parameters:
        criterion = nn.CrossEntropyLoss()
        # opt = optimizer.Adam(net.parameters(), lr=lr, weight_decay=decay)
        opt = optimizer.Adam(net.parameters(), lr=lr)
        # LR_sch = optimizer.lr_scheduler.MultiStepLR(opt, [4,8], gamma=0.1, last_epoch=-1)
        LR_sch = optimizer.lr_scheduler.ReduceLROnPlateau(opt, 'min',patience=15, verbose=True, min_lr=1e-3)
        count = 0
        running_loss = []
        running_top1_acc = []
        running_top2_acc = []
        running_top3_acc = []
        
        #allowed_batches = np.floor((n*3500)/batch_size)
        for epoch in range(num_epochs):
            # print('Training loss', + str(running_loss[-1]))
            print('Start validation')
            ave_top1_acc = 0
            ave_top2_acc = 0
            ave_top3_acc = 0
            ind_ten = t.as_tensor([0, 1, 2], device='cuda:0')
            top1_pred_out = []
            top2_pred_out = []
            top3_pred_out = []
            total_count = 0
            for val_count, (imgs, labels) in enumerate(val_loader):
                net.eval()
                imgs = torch.where(imgs<0.01, 0.0, 1.0)
                x = imgs.cuda()
                opt.zero_grad()
                labels = labels.cuda()
                total_count += labels.size(0)
                out = net.forward(x)
                sorted_out = t.argsort(out, dim=1, descending=True)

                top_1_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:1])
                top1_pred_out.append(top_1_pred.cpu().numpy())


                top_2_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:2])
                top2_pred_out.append(top_2_pred.cpu().numpy())


                top_3_pred_0 = t.index_select(sorted_out, dim=1, index=ind_ten[0])
                top_3_pred_1 = t.index_select(sorted_out, dim=1, index=ind_ten[1])
                top_3_pred_2 = t.index_select(sorted_out, dim=1, index=ind_ten[2])
                top_3_pred = t.index_select(sorted_out, dim=1, index=ind_ten)
                tmp = [top_3_pred_0.item(), top_3_pred_1.item(), top_3_pred_2.item() ]
                top3_pred_out.append(tmp) 
                reshaped_labels = labels.reshape((labels.shape[0], 1))
                
                tiled_2_labels = reshaped_labels.repeat(1, 2)
                tiled_3_labels = reshaped_labels.repeat(1, 3)
  
                batch_top1_acc = t.sum(top_1_pred == labels, dtype=t.float32)
                batch_top2_acc = t.sum(top_2_pred == tiled_2_labels, dtype=t.float32)
                batch_top3_acc = t.sum(top_3_pred == tiled_3_labels, dtype=t.float32)
                
                ave_top1_acc += batch_top1_acc.item()
                ave_top2_acc += batch_top2_acc.item()
                ave_top3_acc += batch_top3_acc.item()
            print("total test examples are", total_count)
            
            running_top1_acc.append(ave_top1_acc / total_count)
            running_top2_acc.append(ave_top2_acc / total_count)
            running_top3_acc.append(ave_top3_acc / total_count)  # (batch_size * (count_2 + 1)))

            print('Average Top-1 accuracy {}'.format( running_top1_acc[-1]))
            print('Average Top-2 accuracy {}'.format( running_top2_acc[-1]))
            print('Average Top-3 accuracy {}'.format( running_top3_acc[-1]))
            print("Saving the predicted value in a csv file")
            tmp = [running_top2_acc[-1],running_top3_acc[-1]]
            val_acc.append(tmp)
            

            with open(directory +"/saved_analysis_file/top1_pred_beam_val_after_%sth_epoch.csv"%(epoch+1), "w", newline="") as f:
                writer = csv.writer(f)
                writer.writerows(zip(top1_pred_out))


            with open(directory + "/saved_analysis_file/top2_pred_beam_val_after_%sth_epoch.csv"%(epoch+1), "w", newline="") as f:
                writer = csv.writer(f)
                writer.writerows(zip(top2_pred_out))  

            with open(directory + "saved_analysis_file/top3_pred_beam_val_after_%sth_epoch.csv"%(epoch+1), "w", newline="") as f:
                writer = csv.writer(f)
                writer.writerows(zip(top3_pred_out))    

            net_name = 'modelafter' +str(epoch) + 'epoch'
            t.save(net.state_dict(), net_name)               

            # LR_sch.step(L)
        top_1[0,idx] = running_top1_acc[-1]
        top_2[0,idx] = running_top2_acc[-1]
        top_3[0,idx] = running_top3_acc[-1]
with open(directory + "val_acc_32_beams.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(val_acc)      
net_name = directory +'checkpoint/img_beam_pred_ckpt'
t.save(net.state_dict(), net_name)



```````````````````````````````````````````````````````
Training size is 1
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
            Conv2d-2           [-1, 16, 10, 10]           2,416
            Linear-3                  [-1, 120]          48,120
            Linear-4                   [-1, 84]          10,164
            Linear-5                   [-1, 64]           5,440
Total params: 66,296
Trainable params: 66,296
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.25
Estimated Total Size (MB): 0.31
----------------------------------------------------------------
Start validation
total test examples are 86
Average Top-1 accuracy 0.38372093023255816
Average Top-2 accuracy 0.7325581395348837
Average Top-3 accuracy 0.8720930